# Agent based modelling of social systems

### Modelling social awereness through a cognitive architecture model
Questions/Topics : 
* What is our question(clear formulation)-> Global awereness
* Topology of network
* Network generator (random?)
* Interaction of agents
* Literature
* Assumptions

### Structure
* Output: Plot Magnitude of event vs Importance of agent (color:fraction of agents, prob. to pass threshold)
* Model: 
 * P(k) distribution of #of outward lines
 * Interaction as a weighted relation of importance of agent and magnitude of event)
* Network: 
 * Random neighbour selection
 * % of neighbours interaction in each time step
* Events
 * Prob. distribution of events log(magn)= $\beta$ - $\gamma$log(occurences)
 * Decay as an exponential function
 * Global magnitude state of each event

In [50]:
import numpy as np
import igraph
import matplotlib.pyplot as plt
import random
from matplotlib import animation
from IPython.display import *
from PIL import Image
#%matplotlib inline

In [51]:
class SocialNet:
    def __init__(self, N):
        """
        Initialize the Graph with N number of agents and 
        give them a number of connections(links) from a 
        power law distribution -> scale-free network. Adjust 
        the size of each vertex to represent the number of
        links it has.
        """
        
        self.time=0
        self.N = N        
        self.g = igraph.Graph(self.N ,directed=True)
        self._create_agents()
        self.visual_style = {}
        self.visual_style["edge_arrow_size"] = 0.5
        layout = self.g.layout("lgl")
        self.visual_style["layout"] = layout
        self.visual_style["vertex_size"]=self.g.vs["size"]
        
        # All events stored in this list! First event is the null-event. All agents start with that.
        self.events=[Event(magnitude=0, t=0)]
    
    def _create_agents(self):
        """Sets attributes of vertices in Graph g"""
        for v in self.g.vs:
            
            # Initialize with null-event
            v["event_id"]=0
            v["event_magnitude"]=0
            
            # Generate number of links and format vertex based on that
            links = self.generate_followers() #self.scale_free_distr() 
            v["size"] = 5 + 3*np.log(links)
            v["links"] = links
            
            if links > 5:
                v["type"] = "core"
                v["color"] = "slate blue"
            else:
                v["type"] = "peripheral"
                v["color"] = "SeaGreen2"  
            
            # Create actual links between agents 
            while links:
                source = v.index
                target = np.random.randint(0, len(self.g.vs))
                if not self.g.are_connected(source, target) and target!=source:
                    self.g.add_edge(source,target)
                    links-=1
                #self.g=self.g.simplify()
        
        for v in self.g.vs:
            v["importance"]= self.g.betweenness(v)
        return
    
    def generate_followers(self):
        chance=np.random.uniform(0,1)
        if chance>10**(-2.5):
            followers=10**( -0.88*np.log10(chance))
        elif chance>10**(-3):
            followers=10**( -2*np.log10(chance) -2.8)
        elif chance>10**(-5):
            followers=10**( -0.2*np.log10(chance)+2.6)
        else:
            followers=len(self.g.vs)-1
        return int(max(np.round(min(followers,len(self.g.vs)-1)),1))
    
    def scale_free_distr(self):
        """Power law distr P(k)=k^-2. In order to 
        limit the number of links possible distr is truncated
        at 0.001
        """
        p = np.random.rand()
        if p <= 0.001:
            return 1
        else:
            return int(np.round(1/np.sqrt(p)))
        
    def update(self):
        """Update all vertices by propagating the information and setting the new state"""
        # In each time-step events happen to agents.
        for v in self.g.vs:
            new_event = Event(magnitude=None, agent_importance=v["importance"], t=self.time)
            # See whether new event is more interesting than old one
            if new_event.magnitude > v["event_magnitude"]:
                self.events.append(new_event)
                # event id is stored in Event object too
                self.events[-1].id = len(self.events)-1
                v["event_id"]=len(self.events)-1
                v["event_magnitude"]=new_event.magnitude    
                v["color"]=new_event.color
        
        # In each time-step agents communicate.
        """ Dynamics: event taken over deterministicly if it is larger event. Important: one-way comparison! """
        for v in self.g.vs:
            for neighbor in v.neighbors(mode=igraph.OUT):
                # only broadcast to a given proportion of neighbors
                if np.random.uniform(0,1)<0.5:
                    if neighbor["event_magnitude"]<v["event_magnitude"]:
                        neighbor["event_magnitude"]=v["event_magnitude"]
                        neighbor["event_id"]=v["event_id"]
                        neighbor["color"]=v["color"]
                # else do nothing    
               
        # At the end of each time-step decay all events and update all vertice event magnitudes
        # Also, calculate the total number of nodes thinking of event i and update event.awareness
        for single_event in self.events:
            single_event.decay()
            num = sum([v==single_event.id for v in self.g.vs["event_id"]])
            if num >= single_event.awareness:
                single_event.awareness = num
                single_event.final_time = self.time+1 #update final time
            
        for v in self.g.vs:
            v["event_magnitude"]=self.events[v["event_id"]].magnitude
                
        self.time+=1       
   
       
        
    def draw(self):        
        igraph.plot(self.g, "img/temp%i.png"%self.time, **self.visual_style)
        #display(Image(filename='img/temp%i.png'%self.time))
        
    """def displayPlot(self):
        plt.savefig("img/temp_plot.png")
        display(Image(filename='img/temp_plot.png'))"""

class Agent(igraph.Vertex):
    def __init__(self, name, num_links):
        self.attr = self.attributes()
        self.num_links = num_links
        self.visual_style["vertex_size"]=2*self.num_links
        self.visual_style["edge_arrow_size"] = 0.5
        self.event = 0
    def _create_links(self):
        pass
    def set_attributes(self):
        self.attr["size"] = int(self.num_links*2)
        
class Event:
    """ Single event. """
    def __init__(self,magnitude=None, agent_importance=1, t=0):
        self.id = None
        # This part is so that if I don't give any value, then the magnitude is generated
        if magnitude==None:
            self.initial_magnitude = np.random.power(a=0.02)
        else:
            self.initial_magnitude = magnitude
        self.magnitude = self.initial_magnitude
        self.color= "#%06x" % random.randint(0, 0xFFFFFF)
        self.agent_importance = agent_importance # each event contains initial agent importance
        self.awareness = 1 #max nr of agents thinking of event at a single time step
        self.init_time = t #time of event creation
        self.final_time = t # time of event peak
        
    def decay(self):
        # keep dt=1 for now since we have discrete time-steps
        gamma=0.0001
        dt=1
        self.magnitude=self.magnitude*(1-gamma*dt)

In [52]:
Net=SocialNet(200)
maxtime=300
for i in range(maxtime):
    Net.update()
    print("updated",i)
    Net.draw()
    print("drawn", i)
    

t=1
fig = plt.figure()
img0 = Image.open('img/temp%i.png'%t)
im=plt.imshow(img0)
def updatefig(fake):
    global t
    img = Image.open('img/temp%i.png'%((t%maxtime)+1))
    im.set_data(img)
    t+=1
    return im
ani = animation.FuncAnimation(fig, updatefig, interval=300)
plt.show()

updated 0 0
drawn 0 0
updated 0 1
drawn 0 1
updated 0 2
drawn 0 2
updated 0 3
drawn 0 3
updated 0 4
drawn 0 4
updated 0 5
drawn 0 5
updated 0 6
drawn 0 6
updated 0 7
drawn 0 7
updated 0 8
drawn 0 8
updated 0 9
drawn 0 9
updated 0 10
drawn 0 10
updated 0 11
drawn 0 11
updated 0 12
drawn 0 12
updated 0 13
drawn 0 13
updated 0 14
drawn 0 14
updated 0 15
drawn 0 15
updated 0 16
drawn 0 16
updated 0 17
drawn 0 17
updated 0 18
drawn 0 18
updated 0 19
drawn 0 19
updated 0 20
drawn 0 20
updated 0 21
drawn 0 21
updated 0 22
drawn 0 22
updated 0 23
drawn 0 23
updated 0 24
drawn 0 24
updated 0 25
drawn 0 25
updated 0 26
drawn 0 26
updated 0 27
drawn 0 27
updated 0 28
drawn 0 28
updated 0 29
drawn 0 29
updated 0 30
drawn 0 30
updated 0 31
drawn 0 31
updated 0 32
drawn 0 32
updated 0 33
drawn 0 33
updated 0 34
drawn 0 34
updated 0 35
drawn 0 35
updated 0 36
drawn 0 36
updated 0 37
drawn 0 37
updated 0 38
drawn 0 38
updated 0 39
drawn 0 39
updated 0 40
drawn 0 40
updated 0 41
drawn 0 41
updated 0 42

In [26]:
fraction = np.asarray([e.agent_importance for e in Net.events])
plt.hist(fraction, bins=300)
plt.ylim([0,10])
plt.show()

In [71]:
def analyze_events(events_list):
    magnitude = np.asarray([e.initial_magnitude for e in events_list if e.awareness>20])
    importance = np.asarray([e.agent_importance for e in events_list if e.awareness>20])
    awareness = np.asarray([e.awareness for e in events_list if e.awareness>20])
    colors = awareness/np.max(awareness)
    plt.scatter(importance, magnitude, c=colors, alpha=0.7, s=40)
    #plt.xscale('log')
    #plt.yscale('log')
    plt.colorbar()
    #plt.ylim([0,0.1])
    plt.show()
    return 

In [72]:
#plt.hist(np.asarray([e.initial_magnitude for e in Net.events]))
#plt.show()
analyze_events(all_events)

In [70]:
import copy
instances=100
network_size=100
maxtime=50

all_events=[]
for j in range(instances):
    Net=SocialNet(network_size)
    for i in range(maxtime):
        Net.update()
        print("updated", j,i)
        Net.draw()
        print("drawn", j,i)
    all_events.extend(copy.deepcopy(Net.events[network_size:]))
analyze_events(all_events)

updated 0 0
drawn 0 0
updated 0 1
drawn 0 1
updated 0 2
drawn 0 2
updated 0 3
drawn 0 3
updated 0 4
drawn 0 4
updated 0 5
drawn 0 5
updated 0 6
drawn 0 6
updated 0 7
drawn 0 7
updated 0 8
drawn 0 8
updated 0 9
drawn 0 9
updated 0 10
drawn 0 10
updated 0 11
drawn 0 11
updated 0 12
drawn 0 12
updated 0 13
drawn 0 13
updated 0 14
drawn 0 14
updated 0 15
drawn 0 15
updated 0 16
drawn 0 16
updated 0 17
drawn 0 17
updated 0 18
drawn 0 18
updated 0 19
drawn 0 19
updated 0 20
drawn 0 20
updated 0 21
drawn 0 21
updated 0 22
drawn 0 22
updated 0 23
drawn 0 23
updated 0 24
drawn 0 24
updated 0 25
drawn 0 25
updated 0 26
drawn 0 26
updated 0 27
drawn 0 27
updated 0 28
drawn 0 28
updated 0 29
drawn 0 29
updated 0 30
drawn 0 30
updated 0 31
drawn 0 31
updated 0 32
drawn 0 32
updated 0 33
drawn 0 33
updated 0 34
drawn 0 34
updated 0 35
drawn 0 35
updated 0 36
drawn 0 36
updated 0 37
drawn 0 37
updated 0 38
drawn 0 38
updated 0 39
drawn 0 39
updated 0 40
drawn 0 40
updated 0 41
drawn 0 41
updated 0 42

In [68]:
a=np.linspace(0,100,20)
a[1:]

array([   5.26315789,   10.52631579,   15.78947368,   21.05263158,
         26.31578947,   31.57894737,   36.84210526,   42.10526316,
         47.36842105,   52.63157895,   57.89473684,   63.15789474,
         68.42105263,   73.68421053,   78.94736842,   84.21052632,
         89.47368421,   94.73684211,  100.        ])